In [1]:
# initialize project:
using Pkg; Pkg.activate("./MyProject");;
include("libs/local_clifford_group.jl");;

using GAP
const g = GAP.Globals;;

  Activating project at `~/GitHub/LocalLambda/MyProject`


# Update rules:

Consider a vertex $A$ of $\Lambda_n^\ell$ given by $$A = \frac{1}{2^n}\sum_{a\in E_n}\alpha_a T_a~.$$ Suppose we perform a measurement of $T_b$ where $b = b_1\oplus \bar 0$, where $b_1 = (b_x,b_z)$ and $\bar 0 \in \mathbb{Z}_2^{2(n-1)}$ is the all zero vector. After conjugation we have $$\Pi_b^r A \Pi_b^r = \bar{\Pi_{b_1}^r}\otimes \bar A$$ where $$\bar A = \frac{1}{2^{n-1}}\sum_{\bar a\in E_{n-1}}\tilde \alpha_a T_a.$$
The new coefficients are given by $$\tilde \alpha_a = \frac{\alpha_{(0,\bar a)}+(-1)^r\alpha_{(b_1,\bar a)}}{\alpha_{(0,\bar 0)}+(-1)^r\alpha_{(b_1,\bar 0)}}.$$ Note that this gives $\tilde \alpha_0 = 1$, as expected. We now demonstrate a counter-example showing that $\bar A$ is not a vertex of $\Lambda_{n-1}^\ell$.

In [8]:
LC2 = LocalCliffordGroup(2);
LC3 = LocalCliffordGroup(3);

In [6]:
# local stabilizer state:
Plus = Vector{Int}([1,1,0,0])
PlusPlus = kron(Plus,Plus);
PPP = kron(PlusPlus,Plus);

In [9]:
# Stabilizer states:
Stab2 = local_clifford_orbit_of_point(LC2,PlusPlus);
Stab3 = local_clifford_orbit_of_point(LC3,PPP);

In [10]:
A2 = Matrix{Rational{Int64}}(undef,0,16)
for s in Stab2; A2 = vcat(A2,transpose(s)); end

A3 = Matrix{Rational{Int64}}(undef,0,64)
for s in Stab3; A3 = vcat(A3,transpose(s)); end

In [12]:
ps3 = PauliString(3).pauli_strings;

In [13]:
# From LRS data (see L3loc.ext in data folder):
v_string = "1  1/3  0  2/3  1  1/3  0  2/3  1/3 -1/3  2/3  0  0 -2/3 -1/3  1/3  1/3 -1/3  0  0  1/3 -1/3  0  0 -1/3 -1  2/3 -2/3  0 -2/3 -1  1/3  1/3 -1/3  0  0  1/3 -1/3  0  0 -1/3 -1  2/3 -2/3  0 -2/3  1/3  1/3  1/3 -1/3 -2/3  0  1/3 -1/3 -2/3  0 -1/3 -1  0 -2/3  2/3  0  1/3  1"
v_split = split(v_string," ")
V = [parse(Rational{Int64},c) for c in v_split if c != ""]

64-element Vector{Rational{Int64}}:
   1
  1//3
   0
  2//3
   1
  1//3
   0
  2//3
  1//3
 -1//3
   ⋮
   0
 -1//3
  -1
   0
 -2//3
  2//3
   0
  1//3
   1

In [18]:
# Check that it is a vertex in LambdaLocal:
NV = findall(x->x<0,A3*V//8);    # Check if any negatives:
ZV = findall(x->x==0,A3*V//8);   # Find tight inequalities
RV = rank(A3[ZV,2:end]);
println("Number of negatives: $(length(NV)), number of tight inequalities: $(length(ZV)), rank of V: $(RV)")

Number of negatives: 0, number of tight inequalities: 115, rank of V: 63


In [22]:
# Paulis that commute with XII:
VX1 = Vector{Rational{Int64}}([]);
for i in 1:64
    if ps3[i][1] in ['I','X']
        push!(VX1,V[i])
        println("Pauli component $(ps3[i]): $(V[i])")
    end
end

Pauli component III: 1//1
Pauli component IIX: 1//3
Pauli component IIY: 0//1
Pauli component IIZ: 2//3
Pauli component IXI: 1//1
Pauli component IXX: 1//3
Pauli component IXY: 0//1
Pauli component IXZ: 2//3
Pauli component IYI: 1//3
Pauli component IYX: -1//3
Pauli component IYY: 2//3
Pauli component IYZ: 0//1
Pauli component IZI: 0//1
Pauli component IZX: -2//3
Pauli component IZY: -1//3
Pauli component IZZ: 1//3
Pauli component XII: 1//3
Pauli component XIX: -1//3
Pauli component XIY: 0//1
Pauli component XIZ: 0//1
Pauli component XXI: 1//3
Pauli component XXX: -1//3
Pauli component XXY: 0//1
Pauli component XXZ: 0//1
Pauli component XYI: -1//3
Pauli component XYX: -1//1
Pauli component XYY: 2//3
Pauli component XYZ: -2//3
Pauli component XZI: 0//1
Pauli component XZX: -2//3
Pauli component XZY: -1//1
Pauli component XZZ: 1//3


In [23]:
norm_0 = (1+1//3); norm_1 = (1-1//3);

In [24]:
VX1_0 = [(VX1[i]+VX1[i+16])/norm_0 for i in 1:16]

16-element Vector{Rational{Int64}}:
   1
   0
   0
  1//2
   1
   0
   0
  1//2
   0
  -1
   1
 -1//2
   0
  -1
  -1
  1//2

In [26]:
# Check that it is a vertex in LambdaLocal:
NV = findall(x->x<0,A2*VX1_0//4);    # Check if any negatives:
ZV = findall(x->x==0,A2*VX1_0//4);   # Find tight inequalities
RV = rank(A2[ZV,2:end]);
println("Number of negatives: $(length(NV)), number of tight inequalities: $(length(ZV)), rank of V: $(RV)")

Number of negatives: 0, number of tight inequalities: 16, rank of V: 14


In [25]:
VX1_1 = [(VX1[i]-VX1[i+16])/norm_1 for i in 1:16]

16-element Vector{Rational{Int64}}:
 1
 1
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 0

In [27]:
# Check that it is a vertex in LambdaLocal:
NV = findall(x->x<0,A2*VX1_1//4);    # Check if any negatives:
ZV = findall(x->x==0,A2*VX1_1//4);   # Find tight inequalities
RV = rank(A2[ZV,2:end]);
println("Number of negatives: $(length(NV)), number of tight inequalities: $(length(ZV)), rank of V: $(RV)")

Number of negatives: 0, number of tight inequalities: 22, rank of V: 14
